# 1. Radar
Por: Camila Lopes ([cclopes.me](https://cclopes.me/))  


Neste módulo, iremos aprender a lidar com dados de radares meteorológicos usando a linguagem Python, passando por leitura, visualização e processamento.


In [1]:
# Importando as bibliotecas necessárias
import pyart


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

/home/cclopes/miniconda3/envs/tutoriais-python/lib/python3.8/site-packages/wradlib/io/radolan.py:939: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  "add_offset": np.float(0),
/home/cclopes/miniconda3/envs/tutoriais-python/li

## 1.2. Visualizando dados de radar

Agora que sabemos como ler diferentes formatos de dados de radar (seção 1.1), vamos aprender a visualizar esses dados com as bibliotecas Py-ART e wradlib.


### 1.2.1. Py-ART

Como explicado anteriormente, o Py-ART possui 4 classes de variáveis com dados de radar: [`Radar`](https://arm-doe.github.io/pyart/API/generated/pyart.core.Radar.html#pyart.core.Radar), [`Grid`](https://arm-doe.github.io/pyart/API/generated/pyart.core.Grid.html#pyart.core.Grid), [`HorizontalWindProfile`](https://arm-doe.github.io/pyart/API/generated/pyart.core.HorizontalWindProfile.html#pyart.core.HorizontalWindProfile) ou [`RadarSpectra`](https://arm-doe.github.io/pyart/API/generated/pyart.core.RadarSpectra.html#pyart.core.RadarSpectra). Para cada classe existem classes e funções correspondentes para a visualização desses dados, que estão no módulo [`pyart.graph`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.html). Essas classes e suas respectivas funções serão explicadas a seguir.

Para todas as classes e funções de visualização de dados, a mesma estrutura é utilizada:

```{python}
# Gerando um display da classe a partir da variável de radar
display = pyart.graph.[classe]([variável de radar])
# Abrindo a figura
fig = plt.figure()
# Gerando o plot
display.[função de plot](field = [campo da variável], [...])
```

Note que a variável de radar não é usada diretamente na função que plota os dados. Os parâmetros das funções de plot podem variar bastante, mas é sempre necessário especificar o campo (refletividade, velocidade doppler, etc) a ser plotado.


#### [`RadarDisplay`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarDisplay.html#pyart.graph.RadarDisplay)

A classe `RadarDisplay` plota dados do tipo `Radar` sem georreferenciamento. As funções de plot disponíveis são:

- [`plot(field, sweep=0)`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarDisplay.plot.html#pyart.graph.RadarDisplay.plot): similar à função `pyart.io.read()` mostrada na seção 1.1.2, essa função identifica o tipo de escaneamento (variável `scan_type`) e plota o campo de radar em uma projeção correspondente à ele (chamando a função `plot_ppi()`, `plot_rhi()` ou `plot_vpt()` explicadas abaixo). Além do campo (`field`), é necessário fornecer a elevação (`sweep`) ou será plotada a primeira elevação por padrão;

- [`plot_ppi(field, sweep=0)`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarDisplay.plot_ppi.html#pyart.graph.RadarDisplay.plot_ppi): plota o campo de radar (`field`) do tipo PPI em uma dada elevação (`sweep`). É possível definir diversos parâmetros como limites de valores (`vmin` e `vmax`), escala de cores (`cmap`), legendas de título (`title`), eixos (`axislabels`) e escala de cores (`colorbar_label`), entre outros;
- [`plot_ray(field, ray)`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarDisplay.plot_ray.html#pyart.graph.RadarDisplay.plot_ray): plota um único feixe
- [`plot_azimuth_to_rhi(field, sweep=0)`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarDisplay.plot_azimuth_to_rhi.html#pyart.graph.RadarDisplay.plot_azimuth_to_rhi): 

- [`plot_rhi(field, sweep=0)`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarDisplay.plot_rhi.html#pyart.graph.RadarDisplay.plot_rhi): 


- [`plot_vpt(field)`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarDisplay.plot_vpt.html#pyart.graph.RadarDisplay.plot_vpt): 

### 1.2.2. wradlib